In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [22]:
df = pd.read_csv('Arrhythmia.csv', sep=';')

In [23]:
df.head()

,age,sex,height,weight,qrs_duration,p-r_interval,q-t_interval,t_interval,p_interval,qrs,T,P,QRST,J,heart_rate,q_wave,r_wave,s_wave,R'_wave,S'_wave,AA,AB,AC,AD,AE,AF,AG,AH,AI,AJ,AK,AL,AM,AN,AO,AP,AR,AS,AT,AU,...,JO,JP,JR,JS,JT,JU,JV,JY,JZ,KA,KB,KC,KD,KE,KF,KG,KH,KI,KJ,KK,KL,KM,KN,KO,KP,KR,KS,KT,KU,KV,KY,KZ,LA,LB,LC,LD,LE,LF,LG,diagnosis
0,75,0,190,80,91,193,371,174,121,-16,13,64,-2,?,63,0,52,44,0,0,32,0,0,0,0,0,0,0,44,20,36,0,28,0,0,0,0,0,0,52,...,0.0,8.4,-10.0,0.0,0.0,0.6,5.9,-3.9,52.7,-0.3,0.0,15.2,-8.4,0.0,0.0,0.9,5.1,17.7,70.7,-0.4,0.0,13.5,-4.0,0.0,0,0.9,3.9,25.5,62.9,-0.3,0.0,9.0,-0.9,0.0,0,0.9,2.9,23.3,49.4,8
1,56,1,165,64,81,174,401,149,39,25,37,-17,31,?,53,0,48,0,0,0,24,0,0,0,0,0,0,0,64,0,0,0,24,0,0,0,0,0,0,32,...,0.0,5.8,-7.7,0.0,0.0,0.9,3.8,-5.7,27.7,-0.2,0.0,9.5,-5.0,0.0,0.0,0.5,2.6,11.8,34.6,-0.4,0.0,11.0,-2.4,0.0,0,0.4,2.6,21.6,43.4,-0.5,0.0,8.5,0.0,0.0,0,0.2,2.1,20.4,38.8,6
2,54,0,172,95,138,163,386,185,102,96,34,70,66,23,75,0,40,80,0,0,24,0,0,0,0,0,0,20,56,52,0,0,40,0,0,0,0,0,0,28,...,0.0,5.8,-4.1,4.0,-0.5,0.4,0.3,20.4,23.3,0.7,0.0,10.0,-5.7,0.0,0.0,0.5,2.2,-3.0,20.7,1.3,0.0,11.1,-3.4,0.0,0,0.4,3.4,11.5,48.2,0.9,0.0,9.5,-2.4,0.0,0,0.3,3.4,12.3,49.0,10
3,55,0,175,94,100,202,380,179,143,28,11,-5,20,?,71,0,72,20,0,0,48,0,0,0,0,0,0,0,64,36,0,0,36,0,0,0,0,0,0,20,...,0.0,9.0,-7.9,0.0,0.0,0.1,4.1,7.6,51.0,0.4,0.0,15.0,-5.5,0.0,0.0,0.1,3.3,28.8,63.1,0.1,0.0,15.2,-3.7,0.0,0,0.6,3.0,36.8,68.0,0.1,0.0,12.2,-2.2,0.0,0,0.4,2.6,34.6,61.6,1
4,75,0,190,80,88,181,360,177,103,-16,13,61,3,?,?,0,48,40,0,0,28,0,0,0,0,0,0,0,40,24,0,0,24,0,0,0,0,0,0,52,...,0.0,8.5,-10.2,0.0,0.0,-1.0,4.7,-4.0,43.0,-0.2,0.0,15.2,-7.8,0.0,0.0,-0.1,4.9,16.2,63.2,-0.2,0.0,9.1,-0.9,0.0,0,-0.2,2.9,21.7,48.9,-0.4,0.0,13.1,-3.6,0.0,0,-0.1,3.9,25.4,62.8,7


In [24]:
df = df.replace('?','0')

In [25]:
dict = {
    'diagnosis':{
        1:0,
        2:1,
        3:1,
        4:1,
        5:1,
        6:1,
        7:1,
        8:1,
        9:1,
        10:1,
        11:1,
        12:1,
        13:1,
        14:1,
        15:1,
        16:0
    }
}   
df=df.replace(dict)
df.head()

,age,sex,height,weight,qrs_duration,p-r_interval,q-t_interval,t_interval,p_interval,qrs,T,P,QRST,J,heart_rate,q_wave,r_wave,s_wave,R'_wave,S'_wave,AA,AB,AC,AD,AE,AF,AG,AH,AI,AJ,AK,AL,AM,AN,AO,AP,AR,AS,AT,AU,...,JO,JP,JR,JS,JT,JU,JV,JY,JZ,KA,KB,KC,KD,KE,KF,KG,KH,KI,KJ,KK,KL,KM,KN,KO,KP,KR,KS,KT,KU,KV,KY,KZ,LA,LB,LC,LD,LE,LF,LG,diagnosis
0,75,0,190,80,91,193,371,174,121,-16,13,64,-2,0,63,0,52,44,0,0,32,0,0,0,0,0,0,0,44,20,36,0,28,0,0,0,0,0,0,52,...,0.0,8.4,-10.0,0.0,0.0,0.6,5.9,-3.9,52.7,-0.3,0.0,15.2,-8.4,0.0,0.0,0.9,5.1,17.7,70.7,-0.4,0.0,13.5,-4.0,0.0,0,0.9,3.9,25.5,62.9,-0.3,0.0,9.0,-0.9,0.0,0,0.9,2.9,23.3,49.4,1
1,56,1,165,64,81,174,401,149,39,25,37,-17,31,0,53,0,48,0,0,0,24,0,0,0,0,0,0,0,64,0,0,0,24,0,0,0,0,0,0,32,...,0.0,5.8,-7.7,0.0,0.0,0.9,3.8,-5.7,27.7,-0.2,0.0,9.5,-5.0,0.0,0.0,0.5,2.6,11.8,34.6,-0.4,0.0,11.0,-2.4,0.0,0,0.4,2.6,21.6,43.4,-0.5,0.0,8.5,0.0,0.0,0,0.2,2.1,20.4,38.8,1
2,54,0,172,95,138,163,386,185,102,96,34,70,66,23,75,0,40,80,0,0,24,0,0,0,0,0,0,20,56,52,0,0,40,0,0,0,0,0,0,28,...,0.0,5.8,-4.1,4.0,-0.5,0.4,0.3,20.4,23.3,0.7,0.0,10.0,-5.7,0.0,0.0,0.5,2.2,-3.0,20.7,1.3,0.0,11.1,-3.4,0.0,0,0.4,3.4,11.5,48.2,0.9,0.0,9.5,-2.4,0.0,0,0.3,3.4,12.3,49.0,1
3,55,0,175,94,100,202,380,179,143,28,11,-5,20,0,71,0,72,20,0,0,48,0,0,0,0,0,0,0,64,36,0,0,36,0,0,0,0,0,0,20,...,0.0,9.0,-7.9,0.0,0.0,0.1,4.1,7.6,51.0,0.4,0.0,15.0,-5.5,0.0,0.0,0.1,3.3,28.8,63.1,0.1,0.0,15.2,-3.7,0.0,0,0.6,3.0,36.8,68.0,0.1,0.0,12.2,-2.2,0.0,0,0.4,2.6,34.6,61.6,0
4,75,0,190,80,88,181,360,177,103,-16,13,61,3,0,0,0,48,40,0,0,28,0,0,0,0,0,0,0,40,24,0,0,24,0,0,0,0,0,0,52,...,0.0,8.5,-10.2,0.0,0.0,-1.0,4.7,-4.0,43.0,-0.2,0.0,15.2,-7.8,0.0,0.0,-0.1,4.9,16.2,63.2,-0.2,0.0,9.1,-0.9,0.0,0,-0.2,2.9,21.7,48.9,-0.4,0.0,13.1,-3.6,0.0,0,-0.1,3.9,25.4,62.8,1


In [26]:
X = df.drop(['diagnosis'], axis=1)
y = df.diagnosis

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [27]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier()
KNN.fit(x_train, y_train)
KNN.score(x_test,y_test)

0.7252747252747253

In [28]:
from sklearn.svm import SVC, LinearSVC
SVM = SVC()
SVM.fit(x_train, y_train)
SVM.score(x_test,y_test)

0.7582417582417582

In [29]:
from sklearn.naive_bayes import GaussianNB
NB = GaussianNB()
NB.fit(x_train, y_train)
NB.score(x_test,y_test)

0.7362637362637363

In [30]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(x_train, y_train)
DT.score(x_test,y_test)

0.7692307692307693

In [31]:
#Chi-Square Test

import scipy.stats as stats
from scipy.stats import chi2

def chi_sqr(test_column,output_column):
    observed_value = pd.crosstab(test_column,output_column)
    #print('observed value:\n',observed_value)
    val = stats.chi2_contingency(observed_value)
    expected_value=val[3]
    #print('expected value:\n',expected_value)
    no_of_rows=len(observed_value.iloc[0:2,0])
    no_of_columns=len(observed_value.iloc[0,0:2])
    dgof=(no_of_rows-1)*(no_of_columns-1)
    alpha=0.05
    #print('degree of freedom:',dgof)
    for o,e in zip(observed_value.values,expected_value):
        chi_2 = sum([(o-e)**2./e])       
    chi2_stat = np.sum(chi_2)
    return chi2_stat
    #critical_value = chi2.ppf(q=1-alpha, df=dgof)
    #print('critical_value:',critical_value)
    
    #p_value=1-chi2.cdf(x=chi2_stat, df=dgof)
    #print(p_value)

In [32]:
df1 = df.drop(['diagnosis'], axis=1)
top_n1 =[]
for i in df1.columns.values:
    top_n1.append(chi_sqr(df1[i],df['diagnosis']))

In [33]:
col_name1 = np.array(X.columns)
a1 = pd.DataFrame(top_n1)
b1 = pd.DataFrame(col_name1)
info1 = pd.concat([a1,b1], axis=1)
info1.columns = ['Score','Features']
info1

,Score,Features
0,0.692884,age
1,6.587175,sex
2,1.443243,height
3,1.443243,weight
4,1.443243,qrs_duration
...,...,...
274,0.000000,LC
275,0.692884,LD
276,0.692884,LE
277,1.443243,LF


In [34]:
top1 = info1.nsmallest(50,'Score')
ft1 = np.array(top1['Features'])
top1
ft1

array(["S'_wave", 'CB', 'CD', 'CS', 'EV', 'EY', 'FF', 'FH', 'FJ', 'FL',
       'FS', 'FZ', 'GA', 'GH', 'IB', 'KP', 'LC', 'LA', 'KF', 'HO', 'AC',
       'AD', 'DI', 'DJ', 'IL', 'IY', 'GD', 'HE', 'GS', 'JT', 'JI', 'AN',
       'BD', 'HC', 'AH', 'AR', 'AS', 'BR', 'EP', 'ES', 'FY', 'GJ', 'CH',
       'HA', 'GN', 'DG', 'HV', 'EU', 'GF', 'HM'], dtype=object)

In [35]:
#Information Gain/Mutual Information

from sklearn.feature_selection import mutual_info_classif

res = mutual_info_classif(X,y,discrete_features=True)

col_name2 = np.array(X.columns)

a2 = pd.DataFrame(res)
b2 = pd.DataFrame(col_name2)
info2 = pd.concat([a2,b2], axis=1)
info2.columns = ['Info_Gain','Features']
info2

,Info_Gain,Features
0,1.356400e-01,age
1,1.625352e-02,sex
2,8.279805e-02,height
3,1.062968e-01,weight
4,1.193862e-01,qrs_duration
...,...,...
274,8.881784e-16,LC
275,3.651774e-02,LD
276,1.410499e-01,LE
277,4.216503e-01,LF


In [41]:
top2 = info2.nlargest(50,'Info_Gain')
ft2 = np.array(top2['Features'])
ft2

array(['KJ', 'JZ', 'JM', 'KU', 'LG', 'JY', 'KI', 'JC', 'HI', 'GY', 'JL',
       'GL', 'IP', 'KT', 'IF', 'HH', 'GV', 'HT', 'LF', 'GK', 'IE', 'IO',
       'JB', 'HS', 'KC', 'JG', 'JR', 'T', 'KM', 'qrs', 'QRST', 'JP', 'IU',
       't_interval', 'q-t_interval', 'JF', 'GO', 'KD', 'JV', 'KZ', 'JK',
       'GE', 'HY', 'KN', 'p_interval', 'HR', 'P', 'p-r_interval', 'KS',
       'KH'], dtype=object)

In [ ]:
#ReliefF

In [42]:
pip install skrebate

  Created wheel for skrebate: filename=skrebate-0.61-cp36-none-any.whl size=29259 sha256=0bd3ae0edf523c2594cfbfaa409dbd02aa5ec48f143c366392a370dde1554e49
  Stored in directory: /root/.cache/pip/wheels/ae/d8/ae/9b51d487e9d02219996d6c260255a216ef07d905b0a0b00ce3
Successfully built skrebate


In [43]:
from sklearn.pipeline import make_pipeline
from skrebate import ReliefF

In [48]:
features, classes = df1.values, df['diagnosis'].values
X_train, X_test, y_train, y_test = train_test_split(features, classes)

fs = ReliefF()
fs.fit(X_train, y_train)

top_n2=[]
for feature_name, feature_score in zip(df.drop('diagnosis', axis=1).columns, fs.feature_importances_):
    top_n2.append(feature_score)

In [49]:
col_name3 = np.array(df.columns)
a3 = pd.DataFrame(top_n2)
b3 = pd.DataFrame(col_name3)
info3 = pd.concat([a3,b3], axis=1)
info3.columns = ['Score','Features']
info3

,Score,Features
0,0.014726,age
1,0.057729,sex
2,0.006174,height
3,0.005972,weight
4,0.047766,qrs_duration
...,...,...
275,0.013043,LD
276,0.084544,LE
277,0.014572,LF
278,0.055318,LG


In [50]:
top3 = info3.nlargest(50,'Score')
ft3 = np.array(top3['Features'])
ft3

array(['LE', 'GJ', 'HR', 'KS', 'KK', 'DD', 'DB', 'GU', 'sex', 'KH', 'LG',
       'QRST', 'heart_rate', 'T', 'IV', 'GY', 'HT', 'GL', 'ID', 'JV',
       'KM', 'qrs_duration', 'KV', 'JZ', 'AJ', 'CO', 'qrs', 'KU', 'IP',
       'DA', 'CJ', 'CL', 'KY', 'DM', 'IH', 'KJ', 'IE', 'KC', 'KA', 'DP',
       'HH', 't_interval', 'GP', 'IO', 'JG', 'GO', 'JY', 'DN', 'JH',
       'q-t_interval'], dtype=object)

In [47]:
#Xvariance
X = df[df.diagnosis == 0]
Y = df[df.diagnosis == 1]
X_var=X[:].var()
Y_var = Y[:].var()
D = X_var + Y_var
DL=D.nlargest(n=50)
ft4=np.array(DL.index)
print (ft4)

['qrs' 'p-r_interval' 'height' 'JZ' 't_interval' 'q-t_interval' 'KJ' 'JY'
 'BN' 'JL' 'CY' 'JM' 'p_interval' 'JB' 'AY' 'KI' 'BI' 'KU' 'DA' 'CL' 'BZ'
 'AV' 'BY' 'FP' 'AU' 'DK' 'JC' 'AJ' 's_wave' 'CK' 'DD' 'r_wave' 'HI' 'BB'
 'BK' 'DM' 'LG' 'GY' 'AI' 'FD' 'CZ' 'DL' 'EO' 'BJ' 'DZ' 'BV' 'EB' 'HH'
 'weight' 'IP']


In [51]:
#CORRELATION
op=y
a1=[]
c1=[]
for i in df1.columns.values:
        df1[i]=df1[i].astype('float64')
        correlation=op.corr(df1[i])
        if correlation>0.1:
            a1.append(correlation)
            c1.append(i)

dfc1=pd.DataFrame(c1)
dfa1=pd.DataFrame(a1)
info5 = pd.concat([dfc1,dfa1], axis=1)
info5.columns = ['name','correlation']
info5

,name,correlation
0,qrs_duration,0.312738
1,t_interval,0.208378
2,s_wave,0.144229
3,AA,0.110948
4,AG,0.113517
5,AJ,0.210947
6,AM,0.106293
7,BB,0.103790
8,BJ,0.195944
9,BL,0.110756


In [57]:
top5 = info5.nlargest(50,'correlation')
ft5= np.array(top5['name'])
ft5

array(['HT', 'qrs_duration', 'HR', 'DB', 'DK', 'DD', 'DZ', 'AJ',
       't_interval', 'JC', 'IV', 'DN', 'HL', 'BJ', 'CJ', 'EO', 'EM', 'JH',
       'CY', 'CC', 'CL', 'HJ', 'HK', 'FT', 'DF', 'FG', 's_wave', 'HS',
       'CO', 'FB', 'HY', 'FA', 'EL', 'IT', 'DP', 'FN', 'BY', 'BN', 'BZ',
       'AG', 'DV', 'DS', 'BU', 'EJ', 'AA', 'FD', 'BL', 'HN', 'EB', 'EC'],
      dtype=object)

In [58]:
from functools import reduce
def top_fs_union(n):
    f1 = ft1[:n]
    f2 = ft2[:n]
    f3 = ft3[:n]
    f4 = ft4[:n]
    f5 = ft5[:n]
    return(reduce(np.union1d, (f1,f2,f3,f4,f5)))

In [61]:
def assemble_run(n,classifier,test_size):
  top_fs=top_fs_union(n)
  top_fs=np.append(top_fs,['diagnosis'], axis=0)
  dfs = df.loc[:,top_fs]
  X = dfs.drop(['diagnosis'], axis=1)
  y = dfs['diagnosis']
  from sklearn.model_selection import train_test_split
  x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=test_size)
  model = classifier()#modify the code ,keep classifier in different function and call here
  model.fit(x_train, y_train)
  print (model.score(x_test,y_test))

In [62]:
#K-NEAREST NEIGHBOUR
#Feature Subset by Union of 5 Best Features from each
KNN5 = assemble_run(5,KNeighborsClassifier,.5)
#Feature Subset by Union of 10 Best Features from each
KNN10 = assemble_run(10,KNeighborsClassifier,.5)
#Feature Subset by Union of 20 Best Features from each
KNN20 = assemble_run(20,KNeighborsClassifier,.5)
#Feature Subset by Union of 30 Best Features from each
KNN30 = assemble_run(30,KNeighborsClassifier,.5)
#Feature Subset by Union of 40 Best Features from each
KNN40 = assemble_run(40,KNeighborsClassifier,.5)
#Feature Subset by Union of 50 Best Features from each
KNN50 = assemble_run(50,KNeighborsClassifier,.5)

0.6592920353982301
0.668141592920354
0.7035398230088495
0.7035398230088495
0.6946902654867256
0.6637168141592921


In [63]:
#SUPPORT VECTOR MACHINE
#Feature Subset by Union of 5 Best Features from each
SVM5  = assemble_run(5,SVC,.5)
#Feature Subset by Union of 10 Best Features from each
SVM10 = assemble_run(10,SVC,.5)
#Feature Subset by Union of 20 Best Features from each
SVM20 = assemble_run(20,SVC,.5)
#Feature Subset by Union of 30 Best Features from each
SVM30 = assemble_run(30,SVC,.5)
#Feature Subset by Union of 40 Best Features from each
SVM40 = assemble_run(40,SVC,.5)
#Feature Subset by Union of 50 Best Features from each
SVM50 = assemble_run(50,SVC,.5)

0.6902654867256637
0.7035398230088495
0.7345132743362832
0.7212389380530974
0.6991150442477876
0.6902654867256637


In [64]:
#DECISION TREE
#Feature Subset by Union of 5 Best Features from each
DT5  = assemble_run(5,DecisionTreeClassifier,.5)
#Feature Subset by Union of 10 Best Features from each
DT10 = assemble_run(10,DecisionTreeClassifier,.5)
#Feature Subset by Union of 20 Best Features from each
DT20 = assemble_run(20,DecisionTreeClassifier,.5)
#Feature Subset by Union of 30 Best Features from each
DT30 = assemble_run(30,DecisionTreeClassifier,.5)
#Feature Subset by Union of 40 Best Features from each
DT40 = assemble_run(40,DecisionTreeClassifier,.5)
#Feature Subset by Union of 50 Best Features from each
DT50 = assemble_run(50,DecisionTreeClassifier,.5)

0.6858407079646017
0.6548672566371682
0.7699115044247787
0.6769911504424779
0.7079646017699115
0.7300884955752213


In [65]:
#NAIVE BAYES
#Feature Subset by Union of 5 Best Features from each
NB5  = assemble_run(5,GaussianNB,.5)
#Feature Subset by Union of 10 Best Features from each
NB10 = assemble_run(10,GaussianNB,.5)
#Feature Subset by Union of 20 Best Features from each
NB20 = assemble_run(20,GaussianNB,.5)
#Feature Subset by Union of 30 Best Features from each
NB30 = assemble_run(30,GaussianNB,.5)
#Feature Subset by Union of 40 Best Features from each
NB40 = assemble_run(40,GaussianNB,.5)
#Feature Subset by Union of 50 Best Features from each
NB50 = assemble_run(50,GaussianNB,.5)

0.8053097345132744
0.7389380530973452
0.7654867256637168
0.7212389380530974
0.7610619469026548
0.7522123893805309
